In [7]:
import os
import requests
from bs4 import BeautifulSoup
import re as re
import json
from selenium import webdriver

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}
session = requests.Session()
session.headers.update(headers)

url = "https://krisha.kz/prodazha/kvartiry/"
r = session.get(url)

In [ ]:
def prepare_entry(r2, id):
    result = []
    columns = []

    #API for price analysis
    url = "https://krisha.kz/analytics/aPriceAnalysis/"
    params = {"id": id}
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://krisha.kz/a/show/762320257"
    }

    r = requests.get(url, params=params, headers=headers)

    soup3 = BeautifulSoup(r.text, "lxml")
    recommended = soup3.select("td.middle")

    rec_price = None
    for i in recommended:
        text = i.get_text()
        #print(text)
        buf = re.search(r"[0-9]+\s[0-9]+", text)
        if buf:
            rec_price = buf.group()

    if rec_price is None:
        return [], []
    
    rec_price = "".join(rec_price.split())
    #Extracting recommended price

    if rec_price:
        result.append(rec_price)
        columns.append("rec_price_m")


    html = r2.text

    soup2 = BeautifulSoup(r2.text, "lxml")
    price = soup2.select_one("div.offer__price").get_text(strip=True)

    if price:
        price = price.replace("\xa0", "")
        price = price.replace("〒", "")
        result.append(price)
        columns.append("price")
    else:
        return [], []
    
    city = soup2.select_one("div.offer__location").get_text(strip=True)

    if city:
        city = re.search(r"^[^,]+", city).group()
        result.append(city)
        columns.append("city")
    else:
        return [], []

    lat = re.search(r'"lat"\s*:\s*([0-9.]+)', html)
    lon = re.search(r'"lon"\s*:\s*([0-9.]+)', html)

    if lat and lon:
        lat = lat.group(1)
        lon = lon.group(1)
        columns.append("lat")
        columns.append("lon")
        result.append(lat)
        result.append(lon)
    else:
        return [], []

    year = soup2.select("div.offer__advert-short-info")
    cnt = 0

    text = r2.text

    bathrooms = re.search(r"Санузел[0-9]", text)

    height = re.search(r"[0-9].[0-9]\sм</dd>", text)

    if bathrooms is None:
        bathrooms = re.search(r"[0-9]\sс/у", text)
        if bathrooms is None:
            bathrooms = '1'
        else:
            bathrooms = bathrooms.group()[0]
    else:
        bathrooms = bathrooms.group()[-1]
    
    if height is None:
        height = 2
    else:
        height = height.group()[:-7]
    
    result.append(bathrooms)
    result.append(height)
    columns.append("bathrooms")
    columns.append("height")


    items = soup2.select("div.offer__info-item")

    text = " ".join(i.get_text(" ", strip=True) for i in items)
    text = text.replace("\xa0", " ")
    text = re.sub(r"\s+", " ", text).strip()

    state = re.search(r"квартиры.*", text, re.I)

    if state is None:
        state = "черновая отделка"
    else:
        state = state.group()[9:]

    result.append(state)
    columns.append("state")

    for i in year:
        cnt += 1
        text = i.get_text(strip=True)
        
        if cnt == 2:
            columns.append("type")
            result.append(text)

        m = re.search(r"\b(19|20)\d{2}\b", text)
        #m = re.search(r"\b(19|20)\d{2}\b", text)
        if m:
            columns.append("year")
            result.append(m.group())
            continue
        area = re.search(r"[0-9]*\sм²", text)
        if area:
            result.append(area.group()[:-3])
            columns.append("area")
            continue
        floor = re.search(r"[0-9]*\sиз", text)
        if floor:
            max_floor = re.search(r"из\s[0-9]*", text)
        
        floor = re.search(r"[0-9]*\sиз", text)
        if floor:
            max_floor = re.search(r"из\s[0-9]*", text).group()
            floor = floor.group()[:-3]
            max_floor = max_floor[3:]
            columns.append("floor")
            columns.append("max_floor")
            result.append(floor)
            result.append(max_floor)
            continue
    
    return result, columns
        


In [10]:
cols = ['rec_price_m', 'price', 'city', 'lat', 'lon', 'bathrooms', 'height', 'state', 'type', 'year', 'floor', 'max_floor', 'area']

In [11]:
import csv
import os

file_exists = os.path.isfile("dataset.csv")

with open("dataset.csv", "a", newline="", encoding="cp1251") as f:
    writer = csv.writer(f)
    if not file_exists:
        writer.writerow(cols)

In [12]:
import time
import random

soup = BeautifulSoup(r.text, "lxml")
cards = soup.select("div.a-card")
#cur = None
max_len = len(cols)

for card in cards:
    id = card["data-id"]
    url = f"https://krisha.kz/a/show/{id}"

    rand = random.randint(0, 500)
    time.sleep(1 + (rand / 1000))

    r2 = session.get(url)
    result, cols = prepare_entry(r2, id)
    
    #print(result, cols)

    if len(result) == max_len:
        print("OK")
        with open("dataset.csv", "a", newline="", encoding="cp1251") as f:
            writer = csv.writer(f)
            writer.writerow(result)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


KeyboardInterrupt: 